In [1]:
%load_ext autoreload

In [58]:
import xlrd
import pandas as pd

from datetime import datetime
from datetime import timedelta, date
from dateutil.relativedelta import relativedelta

from model import plan_var as V
from model import plan_loader as P
from model import reformatter as R

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


from gantt_bar import *

import holidays

In [70]:
ru_holidays = holidays.RU()
datetime.now()

datetime.datetime(2020, 10, 13, 15, 30, 44, 393853)

In [3]:
%autoreload
plan_loader = P.PlanLoader('план.xlsx')
plan_loader.delete_rows_ready()

In [4]:
plan_loader.df.head(2)

,Заливка,Заливка_1,Навеска,Навеска_1,Передача ЭМУ,Сборка щитов,Монтаж СН,Шина УВР/ТР-Р,"Сборка 0,4 кВ","Втор-ка 0,4 кВ",Сборка 6/10 кВ,Втор-ка 6/10 кВ,"Уст-ка 0,4/10кВ",Установ-ка RM-6,Наладка,Комплек-тация,ОТГРУЗКА,ОТГРУЗКА_1
1250,готов,готов,20.11,20-25.11,готов,20-25.11,20-23.11,готов,готов,27-29.11,нет,нет,19.11,20.11,05.12,06.12,2019-12-06,нет
1253,готов,готов,26.11,26-27.11,25.11,26-28.11,25-26.11,готов,готов,02-04.13,нет,нет,25.12,27.12,05.13,06.13,2019-12-10,нет


In [6]:
%autoreload
ref = R.PlanReformatter(plan_loader.df)
gant = ref.get_df_for_gantt()
gant

В зз 1253 столбец Втор-ка 0,4 кВ имеет неверное значение
В зз 1253 столбец Наладка имеет неверное значение
В зз 1253 столбец Комплек-тация имеет неверное значение
В зз 1256 столбец Монтаж СН имеет неверное значение


,Task,Start,Finish,zakaz,Hours
0,Навеска_НМ,2019-11-20 08:20:00,2019-11-20 16:20:00,1250,8
1,Навеска_ВМ,2019-11-20 08:20:00,2019-11-25 16:20:00,1250,48
2,СборкаЩитов,2019-11-20 08:20:00,2019-11-25 16:20:00,1250,48
3,Монтаж_СН,2019-11-20 08:20:00,2019-11-23 16:20:00,1250,32
4,ВторНН,2019-11-27 08:20:00,2019-11-29 16:20:00,1250,24
...,...,...,...,...,...
594,УсткаНН,2020-12-01 08:20:00,2020-12-01 16:20:00,1347,8
595,УсткаВВ,2020-12-01 08:20:00,2020-12-01 16:20:00,1347,8
596,УсткаRM,2020-12-01 08:20:00,2020-12-01 16:20:00,1347,8
597,Наладка,2020-12-03 08:20:00,2020-12-03 16:20:00,1347,8


In [22]:
gant[gant.zakaz=='1250']

,Task,Start,Finish,zakaz,Hours
0,Навеска_НМ,2019-11-20 08:20:00,2019-11-20 16:20:00,1250,8
1,Навеска_ВМ,2019-11-20 08:20:00,2019-11-25 16:20:00,1250,48
2,СборкаЩитов,2019-11-20 08:20:00,2019-11-25 16:20:00,1250,48
3,Монтаж_СН,2019-11-20 08:20:00,2019-11-23 16:20:00,1250,32
4,ВторНН,2019-11-27 08:20:00,2019-11-29 16:20:00,1250,24
5,УсткаНН,2019-11-19 08:20:00,2019-11-19 16:20:00,1250,8
6,УсткаВВ,2019-11-19 08:20:00,2019-11-19 16:20:00,1250,8
7,УсткаRM,2019-11-20 08:20:00,2019-11-20 16:20:00,1250,8
8,Наладка,2019-12-05 08:20:00,2019-12-05 16:20:00,1250,8
9,Комплект,2019-12-06 08:20:00,2019-12-06 16:20:00,1250,8


In [12]:
from plan_model.order import Order
from plan_model.task import Task

In [54]:
%autoreload
ord = Order(gant, '1250')
ord.tasks

[Навеска_НМ,
 Навеска_ВМ,
 СборкаЩитов,
 Монтаж_СН,
 ВторНН,
 УсткаНН,
 УсткаВВ,
 УсткаRM,
 Наладка,
 Комплект]

In [53]:
ord.tasks[9].name

'Комплект'

In [40]:
%autoreload
ord.get_task_by_name('Наладка').end

Timestamp('2019-12-05 16:20:00')

In [36]:
True | False


True

In [169]:
%autoreload
tdh = ref.get_df_timedelta_hours()
tdh.head(2)


,Task,Start,Finish,zakaz,Hours
0,Навеска_ВМ,2020-09-10 08:20:00,2020-09-14 16:20:00,1321,104.0
1,ЭМУ,2020-09-11 08:20:00,2020-09-11 16:20:00,1321,8.0


In [179]:
from dateutil.rrule import rrule, DAILY

In [207]:
%autoreload
V.get_not_working_days()[3]

datetime.date(2020, 5, 2)

In [218]:
t = tdh.iloc[1]
exl_date = [date(2020,9,8)]
t['Hours'] = 0
for d in rrule(DAILY, dtstart=t['Start'], until=t['Finish']):
    if date(d.year,d.month,d.day) not in exl_date:
        print(d)
        t['Hours'] += 8
print(t)


2020-09-11 08:20:00
Task                      ЭМУ
Start     2020-09-11 08:20:00
Finish    2020-09-11 16:20:00
zakaz                    1321
Hours                       8
Name: 1, dtype: object


In [24]:
%autoreload
gant = ref.get_df_for_gantt(num=25)
g = gant[gant['Task'] == 'СборкаВВ']

1313 Навеска_ВМ
1313 СборкаЩитов
2020-08-22 00:00:00
2020-08-23 00:00:00
1313 Монтаж_СН
2020-08-29 00:00:00
1313 ВторНН
1313 ВторВВ
2020-08-22 00:00:00
2020-08-23 00:00:00
1313 УсткаНН
1313 УсткаВВ
1313 Наладка
2020-08-29 00:00:00
2020-08-30 00:00:00
1313 Комплект
1324 Комплект
1325 Комплект
1322 Наладка
1322 Комплект
1320 ВторНН
2020-09-12 00:00:00
2020-09-13 00:00:00
1320 Наладка
1320 Комплект
1321 Навеска_ВМ
2020-09-12 00:00:00
2020-09-13 00:00:00
1321 ЭМУ
1321 СборкаЩитов
2020-09-12 00:00:00
2020-09-13 00:00:00
1321 Монтаж_СН
1321 ШинаУВР
1321 Шина_ТР
1321 СборкаНН
2020-09-12 00:00:00
2020-09-13 00:00:00
1321 ВторНН
2020-09-19 00:00:00
2020-09-20 00:00:00
1321 УсткаНН
1321 УсткаВВ
1321 УсткаRM
1321 Наладка
1321 Комплект
1317 Навеска_НМ
1317 Навеска_ВМ
2020-09-19 00:00:00
2020-09-20 00:00:00
1317 ЭМУ
1317 СборкаЩитов
2020-09-19 00:00:00
2020-09-20 00:00:00
1317 Монтаж_СН
2020-09-26 00:00:00
2020-09-27 00:00:00
1317 ШинаУВР
1317 Шина_ТР
1317 СборкаНН
2020-09-19 00:00:00
2020-09-20 00

In [105]:
fig = go.Figure()
fig.add_trace(go.Box(y = nal_time_sec, boxmean = True))
fig.show()

In [16]:
for _, i in gant.head(5).iterrows():
    print(i)

Task               Заливка_НМ
Start     2020-09-18 08:20:00
Finish    2020-09-23 16:20:00
zakaz                    1333
Name: 0, dtype: object
Task               Заливка_ВМ
Start     2020-09-24 08:20:00
Finish    2020-10-01 16:20:00
zakaz                    1333
Name: 1, dtype: object
Task               Навеска_НМ
Start     2020-10-15 08:20:00
Finish    2020-10-15 16:20:00
zakaz                    1333
Name: 2, dtype: object
Task               Навеска_ВМ
Start     2020-10-15 08:20:00
Finish    2020-10-20 16:20:00
zakaz                    1333
Name: 3, dtype: object
Task                      ЭМУ
Start     2020-10-19 08:20:00
Finish    2020-10-19 16:20:00
zakaz                    1333
Name: 4, dtype: object


In [28]:
l1 = ['Комплект', 'Наладка', 'УсткаRM', 'УсткаВВ', 'УсткаНН', 'ВторВВ', 'СборкаВВ', 'ВторНН', 'СборкаНН', 'Шина_ТР', 'ШинаУВР', 'Монтаж_СН', 'СборкаЩитов', 'ЭМУ', 'Навеска_ВМ', 'Навеска_НМ', 'Заливка_ВМ', 'Заливка_НМ']
l2 = ['Наладка', 'УсткаRM', 'УсткаВВ', 'УсткаНН', 'ВторВВ', 'СборкаВВ', 'ВторНН', 'СборкаНН', 'Шина_ТР', 'ШинаУВР', 'Монтаж_СН', 'СборкаЩитов', 'ЭМУ', 'Навеска_ВМ', 'Навеска_НМ', 'Заливка_ВМ', 'Заливка_НМ']
i=0
while i < len(l1):
            if l1[i] not in l2:
                print(i, l1[i])
                l1.pop(i)
            i+=1

0 Комплект


In [21]:
full_y_list = ref.get_jobs_names()[::-1]
full_y_list

['Комплект',
 'Наладка',
 'УсткаRM',
 'УсткаВВ',
 'УсткаНН',
 'ВторВВ',
 'СборкаВВ',
 'ВторНН',
 'СборкаНН',
 'Шина_ТР',
 'ШинаУВР',
 'Монтаж_СН',
 'СборкаЩитов',
 'ЭМУ',
 'Навеска_ВМ',
 'Навеска_НМ',
 'Заливка_ВМ',
 'Заливка_НМ']

In [29]:
gant[gant['Task'].isin(full_y_list[0:2])]

,Task,Start,Finish,zakaz
14,Наладка,2020-10-28 08:20:00,2020-10-28 16:20:00,1333
15,Комплект,2020-10-30 08:20:00,2020-10-30 16:20:00,1333
27,Наладка,2020-10-25 08:20:00,2020-10-29 16:20:00,1326
28,Комплект,2020-10-29 08:20:00,2020-10-29 16:20:00,1326
31,Наладка,2020-10-29 08:20:00,2020-10-29 16:20:00,1341
32,Комплект,2020-10-29 08:20:00,2020-10-29 16:20:00,1341
47,Наладка,2020-11-05 08:20:00,2020-11-05 16:20:00,1334
48,Комплект,2020-11-05 08:20:00,2020-11-05 16:20:00,1334
63,Наладка,2020-11-06 08:20:00,2020-11-06 16:20:00,1337
64,Комплект,2020-11-09 08:20:00,2020-11-09 16:20:00,1337


In [11]:
%autoreload
dd = data = ref.get_df_for_gantt(2,1100)
dd

NameError: name 'ref' is not defined

In [12]:
%autoreload
fig = go.Figure()
b = GanttBar()

data = ref.get_df_for_gantt(17,10)
for i in range(data.shape[0]):
    fig.add_trace(b.get_bar(data['Start'][i], data['Finish'][i],
                    l.index(data['Task'][i]), group=int(data['zakaz'][i])))
    
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [n for n in range(len(l))],
        ticktext = l
    )
)

fig.update_xaxes(
    dtick='D1',
    range=[dt.today() - timedelta(days=7),
          dt.today() + timedelta(days=30),]
    )
fig.show()

NameError: name 'go' is not defined

In [13]:
dt.today() - timedelta(days=10)

datetime.datetime(2020, 9, 11, 12, 4, 47, 820685)

In [14]:
data[data['Task'] == 'УсткаRM']

NameError: name 'data' is not defined